In [1]:
import os

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description
from datetime import datetime
from dotenv import load_dotenv

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [3]:
#(https://makersuite.google.com/)

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [5]:
def transfer_to_calendar_agent(input_str):
    return calendar_agent_executor.invoke({"input": input_str})
def transfer_to_travel_agent(input_str):
    return travel_agent_executor.invoke({"input": input_str})

In [ ]:
from planing_tools import weatherapi_forecast_periods, query_rag
ddg_search = DuckDuckGoSearchAPIWrapper()

travel_planing_tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="""Essa ferramenta DEVE ser utilizada para buscar eventos relevantes no período fornecido pelo usuário. 
        Ela é útil para obter informações sobre eventos ou atividades especiais que estão acontecendo na cidade de destino nas datas que o usuário informou. 
        O modelo deve usá-la para complementar as sugestões de atividades."""
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="""Essa ferramenta DEVE ser usada *antes* de gerar o roteiro turístico, após o modelo coletar todas as informações necessárias do usuário, incluindo as datas exatas. 
        O modelo deve consultar o clima separadamente para um dia de cada vez no período informado, garantindo que as atividades do roteiro sejam planejadas de acordo com as condições climáticas. 
        A previsão do tempo é um passo obrigatório quando é um fator relevante para o planejamento."""
        ),
    Tool(
        name="Query RAG",
        func=query_rag,
        description="""Esta ferramenta deve ser usada quando o modelo souber a cidade de destino e os interesses do usuário, com o objetivo de fornecer informações sobre pontos turísticos e atrações que se alinham com esses interesses. 
        O modelo deve utilizar essa ferramenta para sugerir atividades e lugares específicos a visitar, baseados na cidade e nos interesses fornecidos."""
    ),
    Tool(
        name="Calendar Agent",
        func=transfer_to_calendar_agent,
        description="Esse agente lida com tudo relacionado ao calendário. As mensagens enviadas a ele devem estar em Português."
    ),
]

In [7]:
from calendar_tools import list_calendar_list, list_calendar_events, insert_calendar_event, create_calendar

google_calendar_tools = [
    Tool(
        name="Create Calendar List",
        func=create_calendar,
        description="""
        Use a função create_calendar_list para criar um novo calendário.
        Forneça o resumo (título) do calendário que você deseja criar.
        - Exemplo de uso: 'Meu Calendário de Viagens'
        - Essa função vai criar um novo calendário com o resumo fornecido.
        """
    ),
    Tool(
        name="List Calendar List",
        func=list_calendar_list,
        description="""
        Use a função list_calendar_list para obter a lista dos seus calendários disponíveis no Google Calendar.
        Se você quiser limitar o número de calendários retornados, forneça o número máximo desejado.
        - Exemplo de uso: Forneça o número máximo de calendários a serem listados, como 50. Se não quiser um limite, apenas diga para listar 200 o máximo por padrão.
        """
    ),
    Tool(
        name="List Calendar Events",
        func=list_calendar_events,
        description="""
        Use a função list_calendar_events para obter a lista de eventos de um calendário específico.
        Você precisa fornecer o ID do calendário e, opcionalmente, o número máximo de eventos a serem listados.
        - Exemplo de uso para o calendário principal: forneça 'primary' como o ID do calendário.
        - Exemplo de uso com limite: forneça o ID do calendário e o número máximo de eventos, como 'primary', 20.
        - Para listar eventos de um calendário específico, primeiro liste seus calendários para encontrar o ID correto.
        """
    ),
    Tool(
    name="Insert Calendar Event",
    func=insert_calendar_event,
    description="""
        Use a função Insert Calendar Event para adicionar **um único evento** a um calendário Google.
        Esta função deve ser chamada **uma vez para cada evento** que precisa ser agendado.

        A entrada para esta função deve ser uma descrição clara e completa de **um único evento**
        em linguagem natural, contendo todas as informações necessárias para agendá-lo individualmente.

        Se o usuário pediu para criar um novo calendário, você DEVE utilizar o calendar_id desse novo calendário para a inserção dos eventos. 
        
        **Formato de entrada esperado:**

        **Descrição em Linguagem Natural (obrigatório):**
        Forneça uma descrição textual detalhada para **um único evento**. Inclua o calendar_id (se fornecido pelo usuário), o que será feito,
        quando (data e hora específicas), onde (local exato) e quaisquer detalhes adicionais relevantes
        para esse evento em particular.

        **Exemplos de descrições de eventos individuais:**
        - "Agendar almoço com a Maria na sexta-feira ao meio-dia no restaurante X."
        - "Criar um compromisso para pagar as contas no dia 10 de janeiro às 9h da manhã."
        - "Bloquear minha agenda para foco no projeto Y na próxima segunda-feira, das 14h às 17h, na minha mesa."

        **Como usar esta função corretamente:**

        1. **Identifique cada evento individualmente:** Se a intenção é agendar múltiplos eventos,
           trate cada um como uma chamada separada para esta função.
        2. **Obtenha o ID do calendário (se necessário):** Se o usuário mencionar um nome de
           calendário específico, use a função 'list_calendar_list' **primeiro** para obter
           o 'calendar_id' correspondente.
        3. **Chame esta função com a descrição de um único evento:** Forneça a descrição completa
           em linguagem natural para o evento específico que você deseja agendar.

        **Informações essenciais para cada descrição de evento:**
        - **O que:** Uma descrição concisa do evento (ex: "Reunião de equipe", "Consulta médica").
        - **Quando:** A data e hora de início e fim **precisas** do evento.
        - **Onde:** O local **específico** do evento.
        - **Fuso Horário (opcional):** Se aplicável, mencione o fuso horário se for diferente
          de "America/Fortaleza".

        **Exemplo de uso**

        *Usuário: Agendar visita ao Aquário Natal sábado das 9h às 12h.*
        *Agente:* (Chama 'Insert Calendar Event' com a Action Input: "Agendar visita ao Aquário Natal sábado das 9h às 12h.")

        **Observação IMPORTANTE:** Esta função **NÃO** deve ser usada para criar múltiplos eventos
        com uma única chamada. Cada chamada deve corresponder a **um único evento**.

        Ao final da execução desta função, você DEVE fornecer ao usuário a confirmação do agendamento
        desse evento específico, incluindo o título, data, hora.

        No final forneça o link para o calendário google: [https://www.google.com/calendar]
    """
    )
]

calendar v3 service created successfully


In [8]:
travel_planing_prompt = hub.pull("tales/agente_turismo")
google_calendar_prompt = hub.pull("tales/agente_calendario")

In [9]:
llm_with_stop = llm.bind(stop=["\nObservation"])
planing_prompt = travel_planing_prompt.partial(
    tools=render_text_description(travel_planing_tools),
    tool_names=", ".join([t.name for t in travel_planing_tools]),
)

In [10]:
calendar_prompt = google_calendar_prompt.partial(
    tools=render_text_description(google_calendar_tools),
    tool_names=", ".join([t.name for t in google_calendar_tools]),
)

In [11]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_64596\275711510.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")


In [12]:
travel_planing_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | planing_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

travel_agent_executor = AgentExecutor(agent=travel_planing_agent, tools=travel_planing_tools, verbose=True, memory=memory)

In [13]:
google_calendar_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | calendar_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

calendar_agent_executor = AgentExecutor(agent=google_calendar_agent, tools=google_calendar_tools, verbose=True, memory=memory)

In [14]:
response = travel_agent_executor.invoke({"input": 'Boa noite'})

print(response["output"])

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Boa noite! Para onde você está planejando viajar?


> Finished chain.
Boa noite! Para onde você está planejando viajar?


In [15]:
response = travel_agent_executor.invoke({"input": 'Eu estou planejando visitar Natal no Rio Grande do Norte'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma excelente escolha.  Quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?


> Finished chain.
Ótimo! Natal é uma excelente escolha.  Quais atividades ou experiências você está buscando nessa viagem? Por exemplo, prefere explorar praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?


In [16]:
response = travel_agent_executor.invoke({"input": 'Eu planejo visitar praias e restaurantes regionais'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Perfeito! Praias e restaurantes regionais são ótimos planos para Natal.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?


> Finished chain.
Perfeito! Praias e restaurantes regionais são ótimos planos para Natal.  Tem um orçamento em mente ou um limite de gastos que gostaria de seguir durante a viagem?


In [17]:
response = travel_agent_executor.invoke({"input": 'Eu ainda não sei'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Tudo bem! Sem problemas.  Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?
```


> Finished chain.
Tudo bem! Sem problemas.  Você segue alguma dieta específica ou tem restrições alimentares, como alergias, que devo levar em consideração?
```


In [18]:
response = travel_agent_executor.invoke({"input": 'Não possuo nenhuma restrição alimentar.'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ok! Sem restrições alimentares, ótimo. Quantos dias você pretende passar em Natal?
```


> Finished chain.
Ok! Sem restrições alimentares, ótimo. Quantos dias você pretende passar em Natal?
```


In [19]:
response = travel_agent_executor.invoke({"input": 'Eu planejo passar 2 dias da sábado dia 25 de janeiro ao domingo dia 26'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025/01/25Previsão para 2025/01/25 em Natal:

Manhã:
Hora: 05:00 - Temp: 25.7°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 80%
Hora: 06:00 - Temp: 26.6°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 75%
Hora: 07:00 - Temp: 27.5°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 71%
Hora: 08:00 - Temp: 28.4°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 67%
Hora: 09:00 - Temp: 29.1°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 64%
Hora: 10:00 - Temp: 29.8°C, Condição: Aguaceiros fracos, Chance de chuva: 100%, Umidade: 61%
Hora: 11:00 - Temp: 30.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 89%, Umidade: 57%

Tarde:
Hora: 13:00 - Temp: 29.9°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 30%, Umidade: 59%
Hora: 14:00 - Temp: 29.7°C, Condição: Sol, Chance de chuva: 0%

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2025/01/26Previsão para 2025/01/26 em Natal:

Manhã:
Hora: 05:00 - Temp: 25.2°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 83%
Hora: 06:00 - Temp: 26.3°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 77%
Hora: 07:00 - Temp: 27.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 71%
Hora: 08:00 - Temp: 28.6°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 65%
Hora: 09:00 - Temp: 29.0°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 64%
Hora: 10:00 - Temp: 29.4°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 62%
Hora: 11:00 - Temp: 29.8°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%, Umidade: 60%

Tarde:
Hora: 13:00 - Temp: 29.3°C, Condição: Possibilidade de chuva irregular, Chance de chuva: 100%

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: Atrações turísticas em Natal, Rio Grande do Norte, focadas em praias e restaurantes regionais.ATRAÇÕES
Em Natal, a vida além da 
praia garante aquário, museus, 
parque aquático, cajueiro 
gigante e estádio de futebol
PARQUE  
DAS DUNAS
Um dos grandes 
charmes do gigantesco 
parque é o Bosque dos 
Namorados, que tem 
área de piquenique, 
playground, restaurante e 
centro de visitantes. Vale 
a pena também percorrer 
as trilhas guiadas do 
parque, com vegetação 
exuberante. Embora 
não tenha praia, oferece 
mirantes com lindas vistas 
panorâmicas para o mar.
FOTOS: SHUTTERSTOCK
AQUÁRIO NATAL
Na Praia da Redinha, já no 
caminho para Genipabu, 
possui cerca de 60 
espécies de animais, como 
moreias, peixes de corais, 
cavalos-marinhos, jacarés e 
até mesmo mamíferos, como 
hipopótamos e jaguatiricas. 
Você pode passar a mão 
nas costas dos tubarões e 
chegar perto dos pinguins.

---

PRAIA A PRAIA
A orla urb

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Com base nas informações que você me forneceu, preparei um roteiro para sua viagem a Natal entre os dias 25 e 26 de janeiro de 2025, focado em praias e restaurantes regionais, considerando a previsão do tempo.  Como a previsão indica chuva em grande parte dos dias, incluí opções de atividades internas caso necessário.

**Roteiro de Viagem para Natal - 25 e 26 de Janeiro de 2025**

**Sábado, 25 de Janeiro de 2025:**

* **Manhã (08:00 - 12:00):**  Considerando a alta probabilidade de chuva pela manhã, sugiro começar o dia com uma visita ao Aquário Natal, na Praia da Redinha.  É uma ótima opção para se proteger da chuva e aprender sobre a vida marinha da região.  Após a visita, caso a chuva cesse, você pode caminhar pela Praia da Redinha e observar os pescadores.
* **Tarde (13:00 - 17:00):**  A previsão indica uma pequena melhora no tempo à tarde. Aproveite para ir à Praia de Ponta Negra.  Mesmo com possibilidade de ch

In [20]:
response = travel_agent_executor.invoke({"input": 'Eu quero que você salve os eventos sugeridos no meu calendário google, no novo calendário chamado Janeiro em Natal 2025.'})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Calendar Agent
Action Input: Por favor, crie um novo calendário no Google Calendar chamado "Janeiro em Natal 2025" e adicione os seguintes eventos:

**Sábado, 25 de Janeiro de 2025:**

* **Manhã (08:00 - 12:00):** Visita ao Aquário Natal (Praia da Redinha).  Alternativa: Caminhada na Praia da Redinha (dependendo do tempo).
* **Tarde (13:00 - 17:00):** Praia de Ponta Negra. Alternativa: Parque das Dunas.
* **Noite (18:00 - 22:00):** Jantar em restaurante regional em Ponta Negra.  Alternativa: Caminhada na orla de Ponta Negra (dependendo do tempo).


**Domingo, 26 de Janeiro de 2025:**

* **Manhã (08:00 - 12:00):** Visita ao Centro Histórico de Natal (museus e igrejas).
* **Tarde (13:00 - 17:00):** Mercado Público da Praia da Redinha (experimente ginga com tapioca). Alternativa: Praia do Meio (dependendo do tempo).
* **Noite (18:00 - 22:00):** Jantar em restaurante regional no Centro Histórico ou próximo à hospedagem. Alternativa

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Create Calendar List
Action Input: Janeiro em Natal 2025{'kind': 'calendar#calendar', 'etag': '"KvIJaBXh0pJHTctqT64toffNYps"', 'id': 'a258385c9ce84f151edb2fe6eaf8844e2e9e4d100496d0c8c5dd7d97d214f487@group.calendar.google.com', 'summary': 'Janeiro em Natal 2025', 'timeZone': 'UTC', 'conferenceProperties': {'allowedConferenceSolutionTypes': ['hangoutsMeet']}}

...

```tool_code
Do I need to use a tool? Yes
Action: Insert Calendar Event
Action Input: Agendar Jantar em restaurante regional no Centro Histórico ou próximo à hospedagem no domingo, 26 de janeiro de 2025, das 18h às 22h. Alternativa: Caminhada pela área da hospedagem (dependendo do tempo). calendar_id: a258385c9ce84f151edb2fe6eaf8844e2e9e4d100496d0c8c5dd7d97d214f487@group.calendar.google.com

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


{'kind': 'calendar#event', 'etag': '"3474312213664000"', 'id': 'd18fqfbn4jam2nsi81q6ohhqdg', 'status': 'confirmed', 'htmlLink': 'https://www.google.com/calendar/event?eid=ZDE4ZnFmYm40amFtMm5zaTgxcTZvaGhxZGcgYTI1ODM4NWM5Y2U4NGYxNTFlZGIyZmU2ZWFmODg0NGUyZTllNGQxMDA0OTZkMGM4YzVkZDdkOTdkMjE0ZjQ4N0Bn', 'created': '2025-01-17T23:21:46.000Z', 'updated': '2025-01-17T23:21:46.832Z', 'summary': 'Jantar em restaurante regional', 'description': 'Jantar em restaurante regional no Centro Histórico ou próximo à hospedagem. Alternativa: Caminhada pela área da hospedagem (dependendo do tempo).', 'location': 'Centro Histórico ou próximo à hospedagem', 'creator': {'email': 'tales.v.m.alves@gmail.com'}, 'organizer': {'email': 'a258385c9ce84f151edb2fe6eaf8844e2e9e4d100496d0c8c5dd7d97d214f487@group.calendar.google.com', 'displayName': 'Janeiro em Natal 2025', 'self': True}, 'start': {'dateTime': '2025-01-26T21:00:00Z', 'timeZone': 'America/Fortaleza'}, 'end': {'dateTime': '2025-01-27T01:00:00Z', 'timeZone': 

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Ótimo!  Os eventos foram adicionados ao seu Calendário Google.  Aproveite sua viagem a Natal!  Se precisar de mais alguma coisa, é só me avisar.


> Finished chain.
Ótimo!  Os eventos foram adicionados ao seu Calendário Google.  Aproveite sua viagem a Natal!  Se precisar de mais alguma coisa, é só me avisar.


In [ ]:
print(response["chat_history"])